In [31]:
from tensorflow.keras.layers import Input, Dropout, GlobalAveragePooling2D, Dense, LeakyReLU, Conv2D, MaxPooling2D,\
      Flatten, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model
from timeit import default_timer as timer
from tensorflow.keras.applications.inception_v3 import InceptionV3
import numpy as np
from sklearn.metrics import classification_report
import const
import os


In [32]:
diverted_images_train_path = os.path.join(const.DATASET_REDUIT, "train")
diverted_images_valid_path = os.path.join(const.DATASET_REDUIT, "valid")
diverted_images_test_path = os.path.join(const.DATASET_REDUIT, "test")

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

In [33]:
batch_size = 16

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

num_classes = train_generator.num_classes

def build_model():
    input_layer = Input(shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model()
#model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 2829 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Epoch 1/20
176/176 [==============================] - 76s 428ms/step - loss: 4.6349 - acc: 0.0885 - mean_absolute_error: 0.0819 - val_loss: 3.0508 - val_acc: 0.0696 - val_mean_absolute_error: 0.0823 - lr: 0.0010
Epoch 2/20
176/176 [==============================] - 4s 22ms/step - loss: 3.0296 - acc: 0.0860 - mean_absolute_error: 0.0817 - val_loss: 2.9667 - val_acc: 0.0946 - val_mean_absolute_error: 0.0816 - lr: 0.0010
Epoch 3/20
176/176 [==============================] - 4s 22ms/step - loss: 2.9468 - acc: 0.1095 - mean_absolute_error: 0.0809 - val_loss: 2.9500 - val_acc: 0.0839 - val_mean_absolute_error: 0.0816 - lr: 0.0010
Epoch 4/20
176/176 [==============================] - 4s 22ms/step - loss: 2.9071 - acc: 0.1130 - mean_absolute_error: 0.0807 - val_loss: 2.8608 - val_acc: 0.1071 - val_mean_absolute_error: 0.0810 - lr: 0.0010
Epoch 5/20
176/176 [===========

In [34]:
#j'augmente le batch size
batch_size = 20

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)
num_classes = train_generator.num_classes

def build_model():
    input_layer = Input(shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model()
#model.summary()

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 2829 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Epoch 1/5
141/141 [==============================] - 4s 28ms/step - loss: 7.1046 - acc: 0.0591 - mean_absolute_error: 0.0827 - val_loss: 3.1373 - val_acc: 0.0482 - val_mean_absolute_error: 0.0831 - lr: 0.0100
Epoch 2/5
141/141 [==============================] - 4s 27ms/step - loss: 3.1327 - acc: 0.0452 - mean_absolute_error: 0.0830 - val_loss: 3.1350 - val_acc: 0.0446 - val_mean_absolute_error: 0.0831 - lr: 0.0100
Epoch 3/5
140/141 [============================>.] - ETA: 0s - loss: 3.1351 - acc: 0.0387 - mean_absolute_error: 0.0831
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
141/141 [==============================] - 4s 27ms/step - loss: 3.1351 - acc: 0.0388 - mean_absolute_error: 0.0831 - val_loss: 3.1337 - val_acc: 0.0464 - val_mean_absolute_error: 0.0832 - lr: 0.0100
Epoch 4/5
141/141 [==============================] - 4s 26ms

In [35]:
#j'utilise imagedatagenerator
batch_size = 20

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    diverted_images_train_path,
    target_size=(224, 224),
    batch_size=batch_size)

num_classes = train_generator.num_classes

def build_model():
    input_layer = Input(shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(rate=0.5)(x)  
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.5)(x) 
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model()
#model.summary()

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 2829 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Found 2829 images belonging to 23 classes.
Epoch 1/5
141/141 [==============================] - 23s 159ms/step - loss: 6.6151 - acc: 0.0541 - mean_absolute_error: 0.0829 - val_loss: 3.1415 - val_acc: 0.0446 - val_mean_absolute_error: 0.0832 - lr: 0.0100
Epoch 2/5
141/141 [==============================] - 22s 157ms/step - loss: 3.1471 - acc: 0.0395 - mean_absolute_error: 0.0832 - val_loss: 3.1456 - val_acc: 0.0250 - val_mean_absolute_error: 0.0831 - lr: 0.0100
Epoch 3/5
141/141 [==============================] - ETA: 0s - loss: 3.1487 - acc: 0.0381 - mean_absolute_error: 0.0832
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
141/141 [==============================] - 22s 155ms/step - loss: 3.1487 - acc: 0.0381 - mean_absolute_error: 0.0832 - val_loss: 3.1362 - val_acc: 0.0429 - val_mean_absolute_error: 0.0832 - lr: 0.0100
Epoch 4/5
1

In [36]:
#modification de l'optimizer avec SGD à la place d'Adam
batch_size = 20

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    diverted_images_train_path,
    target_size=(224, 224),
    batch_size=batch_size)

valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

num_classes = train_generator.num_classes

def build_model():
    input_layer = Input(shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(2048, activation='relu', kernel_regularizer=l2(0.01))(x) 
    x = BatchNormalization()(x)  
    x = Dropout(rate=0.5)(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)  
    x = BatchNormalization()(x) 
    x = Dropout(rate=0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model()
#model.summary()

optimizer = SGD(learning_rate=0.01) 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 2829 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Epoch 1/5
141/141 [==============================] - 23s 157ms/step - loss: 17.4828 - acc: 0.0862 - mean_absolute_error: 0.0806 - val_loss: 17.1917 - val_acc: 0.0929 - val_mean_absolute_error: 0.0803 - lr: 0.0100
Epoch 2/5
141/141 [==============================] - 22s 156ms/step - loss: 16.5525 - acc: 0.0936 - mean_absolute_error: 0.0800 - val_loss: 15.4780 - val_acc: 0.1304 - val_mean_absolute_error: 0.0786 - lr: 0.0100
Epoch 3/5
141/141 [==============================] - 22s 156ms/step - loss: 15.7976 - acc: 0.0918 - mean_absolute_error: 0.0800 - val_loss: 14.9341 - val_acc: 0.1089 - val_mean_absolute_error: 0.0796 - lr: 0.0100
Epoch 4/5
141/141 [==============================] - 22s 159ms/step - loss: 15.0468 - acc: 0.1011 - mean_absolute_error: 0.0794 - val_loss: 14.1103 - val_acc: 0.1393 - val_mean_absolute_error: 0.0796 - lr: 0.0100
Epoch 5/5
29/29 [====

In [37]:
def build_model():
    input_layer = Input(shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(2048, kernel_initializer='he_normal')(x)  
    x = LeakyReLU()(x) 
    x = Dropout(rate=0.5)(x)
    x = Dense(1024, kernel_initializer='he_normal')(x) 
    x = LeakyReLU()(x)  
    x = Dropout(rate=0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model()
#model.summary()

optimizer = Adam(learning_rate=0.01, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=10,  
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Epoch 1/10
141/141 [==============================] - 23s 159ms/step - loss: 3715.6208 - acc: 0.0701 - mean_absolute_error: 0.0809 - val_loss: 503.8104 - val_acc: 0.1107 - val_mean_absolute_error: 0.0773 - lr: 0.0100
Epoch 2/10
141/141 [==============================] - 22s 156ms/step - loss: 533.9200 - acc: 0.0865 - mean_absolute_error: 0.0794 - val_loss: 220.6390 - val_acc: 0.1196 - val_mean_absolute_error: 0.0766 - lr: 0.0100
Epoch 3/10
141/141 [==============================] - 22s 156ms/step - loss: 327.3020 - acc: 0.0904 - mean_absolute_error: 0.0791 - val_loss: 217.4539 - val_acc: 0.0768 - val_mean_absolute_error: 0.0803 - lr: 0.0100
Epoch 4/10
141/141 [==============================] - 22s 155ms/step - loss: 251.5881 - acc: 0.0961 - mean_absolute_error: 0.0786 - val_loss: 146.1829 - val_acc: 0.0804 - val_mean_absolute_error: 0.0800 - lr: 0.0100
Epoch 5/10
141/141 [==============================] - 22s 156ms/step - loss: 200.5815 - acc: 0.0872 - mean_absolute_error: 0.0793 - val

In [38]:
# modèle CNN
batch_size = 20

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    diverted_images_train_path,
    target_size=(224, 224),
    batch_size=batch_size)

valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), 
                                                           batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), 
                                                          batch_size=batch_size)

num_classes = train_generator.num_classes

def build_model():
    input_layer = Input(shape=(224, 224, 3))
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)  
    x = MaxPooling2D(pool_size=(2, 2))(x) 
    x = Conv2D(64, (3, 3), activation='relu')(x)  
    x = MaxPooling2D(pool_size=(2, 2))(x)  
    x = Flatten()(x) 
    x = Dense(128, activation='relu')(x)  
    predictions = Dense(num_classes, activation='softmax')(x)  
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model()
#model.summary()

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 2829 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Epoch 1/5
141/141 [==============================] - 24s 165ms/step - loss: 477.8160 - acc: 0.0406 - mean_absolute_error: 0.0832 - val_loss: 3.1363 - val_acc: 0.0411 - val_mean_absolute_error: 0.0832 - lr: 0.0100
Epoch 2/5
141/141 [==============================] - 29s 203ms/step - loss: 3.1394 - acc: 0.0352 - mean_absolute_error: 0.0832 - val_loss: 3.1360 - val_acc: 0.0446 - val_mean_absolute_error: 0.0832 - lr: 0.0100
Epoch 3/5
141/141 [==============================] - ETA: 0s - loss: 3.1392 - acc: 0.0413 - mean_absolute_error: 0.0832
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
141/141 [==============================] - 27s 188ms/step - loss: 3.1392 - acc: 0.0413 - mean_absolute_error: 0.0832 - val_loss: 3.1358 - val_acc: 0.0429 - val_mean_absolute_error: 0.0832 - lr: 0.0100
Epoch 4/5
141/141 [==============================] -

In [39]:
#modèle sequentiel avec Batchnormalization


def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Epoch 1/10
141/141 [==============================] - 30s 201ms/step - loss: 3.3234 - acc: 0.1339 - mean_absolute_error: 0.0782 - val_loss: 4.7609 - val_acc: 0.1911 - val_mean_absolute_error: 0.0723 - lr: 0.0010
Epoch 2/10
141/141 [==============================] - 26s 186ms/step - loss: 2.6064 - acc: 0.2645 - mean_absolute_error: 0.0710 - val_loss: 2.0890 - val_acc: 0.3536 - val_mean_absolute_error: 0.0633 - lr: 0.0010
Epoch 3/10
141/141 [==============================] - 27s 191ms/step - loss: 2.2484 - acc: 0.3489 - mean_absolute_error: 0.0663 - val_loss: 1.8704 - val_acc: 0.4661 - val_mean_absolute_error: 0.0567 - lr: 0.0010
Epoch 4/10
141/141 [==============================] - 27s 190ms/step - loss: 2.0356 - acc: 0.3891 - mean_absolute_error: 0.0628 - val_loss: 2.2086 - val_acc: 0.3821 - val_mean_absolute_error: 0.0590 - lr: 0.0010
Epoch 5/10
141/141 [==============================] - ETA: 0s - loss: 1.8691 - acc: 0.4236 - mean_absolute_error: 0.0603
Epoch 5: ReduceLROnPlateau redu

In [40]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=40,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Epoch 1/40
141/141 [==============================] - 27s 187ms/step - loss: 3.2012 - acc: 0.1367 - mean_absolute_error: 0.0789 - val_loss: 4.0941 - val_acc: 0.1375 - val_mean_absolute_error: 0.0764 - lr: 0.0010
Epoch 2/40
141/141 [==============================] - 28s 194ms/step - loss: 2.6955 - acc: 0.2342 - mean_absolute_error: 0.0733 - val_loss: 4.0162 - val_acc: 0.2429 - val_mean_absolute_error: 0.0681 - lr: 0.0010
Epoch 3/40
141/141 [==============================] - 26s 186ms/step - loss: 2.4705 - acc: 0.2876 - mean_absolute_error: 0.0695 - val_loss: 1.8545 - val_acc: 0.4125 - val_mean_absolute_error: 0.0574 - lr: 0.0010
Epoch 4/40
141/141 [==============================] - 27s 192ms/step - loss: 2.1891 - acc: 0.3510 - mean_absolute_error: 0.0658 - val_loss: 1.6985 - val_acc: 0.4536 - val_mean_absolute_error: 0.0589 - lr: 0.0010
Epoch 5/40
141/141 [==============================] - 26s 184ms/step - loss: 2.0703 - acc: 0.3902 - mean_absolute_error: 0.0640 - val_loss: 2.0882 - val

In [ ]:
# meilleur modèle avec augmentation des données
batch_size = 16
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), 
                                                  batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
# meilleur modèle sans augmentation des données
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), 
                                                  batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
#augmentation de la complexité du modèle, avec augmentation de données
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)

#j'arrête l'execution, le modèle ne performe pas du tout

In [42]:
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(224, 224, 3)))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])

Found 2829 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Found 575 images belonging to 23 classes.
Epoch 1/10
176/176 [==============================] - 58s 319ms/step - loss: 2.7657 - acc: 0.2609 - mean_absolute_error: 0.0705 - val_loss: 5.4471 - val_acc: 0.0536 - val_mean_absolute_error: 0.0827 - lr: 1.0000e-04
Epoch 2/10
176/176 [==============================] - 54s 306ms/step - loss: 2.2880 - acc: 0.3502 - mean_absolute_error: 0.0642 - val_loss: 6.1232 - val_acc: 0.0821 - val_mean_absolute_error: 0.0814 - lr: 1.0000e-04
Epoch 3/10
176/176 [==============================] - 53s 300ms/step - loss: 2.0940 - acc: 0.3903 - mean_absolute_error: 0.0615 - val_loss: 3.1030 - val_acc: 0.2071 - val_mean_absolute_error: 0.0714 - lr: 1.0000e-04
Epoch 4/10
176/176 [==============================] - ETA: 0s - loss: 1.9443 - acc: 0.4390 - mean_absolute_error: 0.0582

KeyboardInterrupt: 

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(224, 224, 3)))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])
#j'arrête le modèle ne semble pas du tout performant

In [ ]:
#modèle sequentiel avec Batchnormalization

batch_size = 16
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])
history_diverted = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)


test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])